In [ ]:
#Importing libraries

import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
#import Random forest classifiers
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns


In [1]:
import pandas as pd

/tmp/ipykernel_7854/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
labels = pd.read_csv("Galaxy_data/training_solutions_rev1.csv")
labels.shape

(61578, 38)

In [3]:
pd.set_option('display.max_columns',None)
labels=labels.drop(labels[labels['Class1.3'] > 0.5].index)

In [4]:
labels = labels[['GalaxyID', 'Class1.1', 'Class1.2', 'Class6.1']]
labels['Result'] = 'i'
labels.loc[labels['Class1.1'] > 0.6, 'Result'] = 'e'
labels.loc[labels['Class1.2'] > 0.6, 'Result'] = 's'
labels.loc[labels['Class6.1'] > 0.6, 'Result'] = 'o'



In [5]:
labels.head()

,GalaxyID,Class1.1,Class1.2,Class6.1,Result
0,100008,0.383147,0.616853,0.054453,s
1,100023,0.327001,0.663777,0.189149,s
2,100053,0.765717,0.177352,0.000000,e
3,100078,0.693377,0.238564,0.320398,e
4,100090,0.933839,0.000000,0.029383,e


In [6]:
# Filter rows where 'Result' is 'o'
result_o = labels[labels['Result'] == 'i']

# Display the resulting DataFrame
print(result_o)


       GalaxyID  Class1.1  Class1.2  Class6.1 Result
10       100150  0.429378  0.524901  0.037499      i
12       100187  0.448521  0.551479  0.252285      i
13       100204  0.467625  0.531480  0.084531      i
21       100367  0.471429  0.512708  0.430580      i
36       100541  0.445052  0.533256  0.229921      i
...         ...       ...       ...       ...    ...
61562    999753  0.458539  0.541461  0.177392      i
61567    999873  0.531642  0.431270  0.382783      i
61571    999900  0.460239  0.511396  0.273592      i
61572    999936  0.545443  0.454557  0.028557      i
61573    999948  0.510379  0.489621  0.245734      i

[11778 rows x 5 columns]


In [7]:
labels=labels.drop(labels[labels['Result'] == 'i'].index)
print(labels)

       GalaxyID  Class1.1  Class1.2  Class6.1 Result
0        100008  0.383147  0.616853  0.054453      s
1        100023  0.327001  0.663777  0.189149      s
2        100053  0.765717  0.177352  0.000000      e
3        100078  0.693377  0.238564  0.320398      e
4        100090  0.933839  0.000000  0.029383      e
...         ...       ...       ...       ...    ...
61570    999893  0.410999  0.519560  0.801448      o
61574    999950  0.901216  0.098784  0.127644      e
61575    999958  0.202841  0.777376  0.839186      o
61576    999964  0.091000  0.909000  0.864000      o
61577    999967  0.767000  0.140000  0.333000      e

[49756 rows x 5 columns]


In [8]:
galaxy_ids = labels['GalaxyID'].to_numpy()

# Display the array of GalaxyID
print(galaxy_ids)

[100008 100023 100053 ... 999958 999964 999967]


In [9]:
from PIL import Image, ImageOps
from numpy import asarray
import os

folder_path = "Galaxy_data/images_training_rev1/"
output_dict = {}

for galaxy_id in galaxy_ids:
    filename = f"{galaxy_id}.jpg"
    filepath = os.path.join(folder_path, filename)

    if os.path.exists(filepath):
        # load the image and convert into numpy array
        img = Image.open(filepath)
        img_gray = ImageOps.grayscale(img)
        numpydata = asarray(img_gray).flatten()

        # store the numpy array in the dictionary with the galaxy_id as the key
        output_dict[galaxy_id] = numpydata

# Now output_dict contains numpy arrays for all specified galaxy IDs


In [1]:
import numpy as np
x=np.array(list(output_dict.values())).mean()

NameError: name 'output_dict' is not defined

In [22]:
#import csv


# Specify the CSV file path
#csv_file_path = 'galaxy.csv'

# Writing the data to the CSV file
#with open(csv_file_path, 'w', newline='') as csv_file:
#    csv_writer = csv.writer(csv_file)

    # Write the header row with keys as column names
#    header = ['GalaxyID'] + [f'Pixel_{i}' for i in range(len(output_dict[100008].ravel()))]
#    csv_writer.writerow(header)

    # Write data rows
#    for key, value in output_dict.items():
        # Flatten the array to a 1D list for the row
#        row_data = [key] + value.ravel().tolist()
#        csv_writer.writerow(row_data)

#print(f'CSV file has been created at: {csv_file_path}')


KeyboardInterrupt: 

In [21]:
output_dict[100008]

(424, 424)

In [24]:
len(output_dict[100008].ravel())

179776

424.0